<a href="https://colab.research.google.com/github/akumarpandey686/brain_tumor_classification/blob/master/Project_vgg16__.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

Using TensorFlow backend.


In [0]:
#Give dataset path
train_path = '/content/drive/My Drive/MTech/final_dataset/train'
test_path = '/content/drive/My Drive/MTech/final_dataset/test'

In [0]:
IMAGE_SIZE = [224, 224]

In [4]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

58892288/58889256 [==============================] - 6s 0us/step


In [0]:
# existing weights training not required 
for layer in vgg.layers:
  layer.trainable = False

In [6]:
# useful for getting number of classes
folders = glob('/content/drive/My Drive/MTech/final_dataset/train/*')
print(len(folders))

2


In [7]:
x = Flatten()(vgg.output)
prediction = Dense(len(folders), activation='softmax')(x)
model = Model(inputs=vgg.input, outputs=prediction)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [0]:
from keras import optimizers


sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9)
model.compile(loss='binary_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [0]:
# Data Augmentation for training
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [0]:
# Data Augmentation for testing
test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [11]:
# target size should be same as initialied for the image size
train_set = train_datagen.flow_from_directory('/content/drive/My Drive/MTech/final_dataset/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 188 images belonging to 2 classes.


In [12]:
test_set = test_datagen.flow_from_directory('/content/drive/My Drive/MTech/final_dataset/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 75 images belonging to 2 classes.


In [13]:
from datetime import datetime
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau

#lr_scheduler = LearningRateScheduler(lr_schedule)

#lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
#                               cooldown=0,
#                               patience=5,
#                               min_lr=0.5e-6)

#num_epochs = 1000
#num_batch_size = 32

checkpoint = ModelCheckpoint(filepath='mymodel.h5', 
                               verbose=1, save_best_only=True)

callbacks = [checkpoint]

start = datetime.now()

model.fit_generator(
  train_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch=188,
  validation_steps=75,
    callbacks=callbacks ,verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Epoch 1/5


/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:616: UserWarning: The input 3 could not be retrieved. It could be because a worker has died.
  UserWarning)


188/188 [==============================] - 276s 1s/step - loss: 6.2131 - accuracy: 0.5951 - val_loss: 2.7886 - val_accuracy: 0.6203

Epoch 00001: val_loss improved from inf to 2.78856, saving model to mymodel.h5
Epoch 2/5
188/188 [==============================] - 135s 720ms/step - loss: 6.3372 - accuracy: 0.5877 - val_loss: 5.5771 - val_accuracy: 0.6485

Epoch 00002: val_loss did not improve from 2.78856
Epoch 3/5
188/188 [==============================] - 138s 736ms/step - loss: 5.9435 - accuracy: 0.6133 - val_loss: 6.9714 - val_accuracy: 0.6043

Epoch 00003: val_loss did not improve from 2.78856
Epoch 4/5
188/188 [==============================] - 135s 716ms/step - loss: 6.2905 - accuracy: 0.5910 - val_loss: 9.7600 - val_accuracy: 0.6053

Epoch 00004: val_loss did not improve from 2.78856
Epoch 5/5
188/188 [==============================] - 137s 728ms/step - loss: 5.6584 - accuracy: 0.6320 - val_loss: 5.5771 - val_accuracy: 0.6907

Epoch 00005: val_loss did not improve from 2.78856
